In [1]:
import services as svs

/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/floodProbProj/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['./whitebox_tools', '-v']


In [ ]:
# Import tails form an xxcell doc. ]
excell_file = '/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/saint_john_NFL_DTM.xlsx'
listTails = svs.importListFromExelCol(excell_file,'Feuil1','ftp_dtm')
importer = svs.dtmTailImporter(listTails)
importer.impotTailToLocalDir()

In [2]:
import os
desktop = os.path.expanduser("~")
print(desktop)


/Users/abdielfer


In [2]:
transformer = svs.dtmTransformer()

In [ ]:
svs.setWBTWorkingDir('/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodMaps/testZone')

In [ ]:
transformer.computeMosaic('zone_one.tif')

In [ ]:
dtmName = 'zone_one.tif' 
transformer.fixNoDataAndfillDTM(dtmName)
# transformer.computeSlope(dtmName)
# transformer.computeAspect(dtmName)

In [6]:
os.remove(os.path.join('/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodMaps/testZone','aspect_zone_one.tif'))